In [1]:
import numpy as np
import matplotlib.pyplot as plt

arm_mass = 0.2 #kg
arm_length = 0.6 #meter
motor_mass = 0.065

arm_lengths = [arm_length,]*4

total_inertia_tensor = np.zeros((3, 3))

arms_inertia_tensor = np.zeros((3, 3))
arms_inertia_tensor[0, 0] = 2*(1/12)*arm_mass*arm_length**2
arms_inertia_tensor[1, 1] = (1/12)*arm_mass*arm_length**2
arms_inertia_tensor[2, 2] = (1/12)*arm_mass*arm_length**2

motors_inertia_tensor = np.zeros((3, 3))
motors_inertia_tensor[0, 0] = 2*motor_mass*arm_length**2
motors_inertia_tensor[1, 1] = 2*motor_mass*arm_length**2
motors_inertia_tensor[2, 2] = 4*motor_mass*arm_length**2

total_inertia_tensor = arms_inertia_tensor + motors_inertia_tensor

In [2]:
def get_dyn_constants(body_mass):
    prop_radius = 0.1016 # meters
    prop_pitch = 0.1143 # meters
    c_d = 0.09 # airfoil
    body_max_omega_z = 100*(np.pi/180) # radians/second; max body angular velocity about z axis
    
    neutral_buoyancy_pulse = 750 # unitless
    g = 9.81
    k_force = body_mass*g/(4*((neutral_buoyancy_pulse)**2))
    k_torque = 0.5*(prop_radius**4)*prop_pitch*c_d*(body_max_omega_z**2)
    return k_force, k_torque


In [3]:
def get_body_constants():
    
    arm_mass = 0.2 #kg
    arm_length = 0.6 #meter
    motor_mass = 0.065

    arm_lengths = [arm_length,]*4

    total_inertia_tensor = np.zeros((3, 3))

    arms_inertia_tensor = np.zeros((3, 3))
    arms_inertia_tensor[0, 0] = 2*(1/12)*arm_mass*arm_length**2
    arms_inertia_tensor[1, 1] = (1/12)*arm_mass*arm_length**2
    arms_inertia_tensor[2, 2] = (1/12)*arm_mass*arm_length**2

    motors_inertia_tensor = np.zeros((3, 3))
    motors_inertia_tensor[0, 0] = 2*motor_mass*arm_length**2
    motors_inertia_tensor[1, 1] = 2*motor_mass*arm_length**2
    motors_inertia_tensor[2, 2] = 4*motor_mass*arm_length**2

    # This formulation of the moment of inertia tensor places the arms along the
    # x and y axes, but in reality the arms of the QC are at 45 degree angles to
    # the x and y axes of the body frame. SOOOOO... rotate the entire inertia
    # tensor about the z-axis by 45 degrees.

    total_inertia_tensor = arms_inertia_tensor + motors_inertia_tensor
    
    R_z = np.zeros((3, 3))
    R_z[0, 0] = np.cos(np.pi/4)
    R_z[0, 1] = -np.sin(np.pi/4)
    R_z[1, 0] = np.sin(np.pi/4)
    R_z[1, 1] = np.cos(np.pi/4)
    
    rot_inertia_tensor = R_z.dot(total_inertia_tensor)
    
    body_mass = arm_mass*2 + motor_mass*4
    return body_mass, rot_inertia_tensor, arm_lengths

In [4]:
def quat_mult(q1, q2):
    #print(q1, q2)
    #print(q1.shape, q2.shape)
    #q3 = np.zeros((1, 4))
    q3 = np.array([0., 0., 0., 0.,])
    q3[0] = q1[0]*q2[0] - q1[1:].dot(q2[1:])
    q3[1:] = q1[0]*q2[1:] + q2[0]*q1[1:] + np.cross(q1[1:], q2[1:])
    return q3

In [5]:
def quat_inv(q1):
    #q2 = np.zeros((1, 4))
    q2 = np.array([0., 0., 0., 0.,])
    mag = np.linalg.norm(q1)
    q2[0] = q1[0]
    q2[1:] = -1.*q1[1:]
    q2 /= (mag*mag)
    return q2

In [6]:
def calc_omega_dot_body(omega_body, inertia_body, torques_body):
    omega_dot_body = np.linalg.inv(inertia_body).dot(torques_body - np.cross(omega_body, (inertia_body.dot(omega_body))))
    return omega_dot_body

In [7]:
# Expects motor_speeds to be a 4-length array with values of (pulse_width_i - 1000)
def get_torques_body(motor_speeds, arm_lengths, k_force, k_torque):
    #torques_body = np.zeros((1, 3))
    torques_body = np.array([0., 0., 0.,])
    #print("torques_body:", torques_body)
    torques_body[0] = arm_lengths[1]*k_force*(motor_speeds[1]**2) - arm_lengths[2]*k_force*(motor_speeds[2]**2)
    torques_body[1] = arm_lengths[3]*k_force*(motor_speeds[3]**2) - arm_lengths[0]*k_force*(motor_speeds[0]**2)
    torques_body[2] = k_torque*(-1*motor_speeds[0]**2 + motor_speeds[1]**2 - motor_speeds[2]**2 + motor_speeds[3]**2)
    #print("torques_body after stuff:", torques_body)
    return torques_body

In [8]:
# Expects motor_speeds to be a 4-length array with values of (pulse_width_i - 1000)
def get_forces_body(motor_speeds, k_force):
    #forces_body = np.zeros((1, 3))
    forces_body = np.array([0., 0., 0.,])
    forces_body[2] = k_force*sum([s**2 for s in motor_speeds])
    return forces_body

In [9]:
# Expects motor_speeds to be a 4-length array with values of (pulse_width_i - 1000)
def update_orientation_body(
    omega_body_prev,
    q_body_prev,
    dt,
    arm_lengths,
    k_force,
    k_torque,
    inertia_body,
    motor_speeds
):
    torques_body = get_torques_body(motor_speeds, arm_lengths, k_force, k_torque)
    #print("torques:", torques_body)
    omega_dot_body = calc_omega_dot_body(omega_body_prev, inertia_body, torques_body)
    #print("omega_dot_body", omega_dot_body)
    omega_body = omega_body_prev + omega_dot_body*dt
    #print("omega_body_prev:", omega_body_prev)
    #print("omega_body:", omega_body)
    delta_theta = np.linalg.norm(omega_body)
    quat_omega_body = np.array([np.cos(delta_theta*dt/2.)] + list(np.sin(delta_theta*dt/2.)*omega_body/(delta_theta + 1e-8)))
    q_body = quat_mult(q_body_prev, quat_omega_body)
    if np.linalg.norm(q_body) < 0.999:
        q_body += 0.1*(1. - np.linalg.norm(q_body)**2)*q_body
        
    return q_body, omega_body

In [10]:
# Expects motor_speeds to be a 4-length array with values of (pulse_width_i - 1000)
def get_force_inertial(vel_inert_prev, q_body, motor_speeds, k_force, k_damp, body_mass):

    forces_body = get_forces_body(motor_speeds, k_force)
    #print("forces_body:", forces_body)
    forces_body_quat = np.array([0., 0., 0., 0.,])
    forces_body_quat[1:] = forces_body
    net_force_inert = body_mass*np.array([0, 0, -9.8]) + \
        quat_mult(quat_mult(quat_inv(q_body), forces_body_quat), q_body)[1:] - \
        k_damp*vel_inert_prev
    return net_force_inert

In [11]:
# Expects motor_speeds to be a 4-length array with values of (pulse_width_i - 1000)
def update_pos_vel_inertial(
    pos_inert_prev,
    vel_inert_prev,
    accel_inert_prev,
    q_body,
    motor_speeds,
    k_force,
    k_damp,
    body_mass,
    dt
):
    
    # Velocity verlet
    pos_inert = pos_inert_prev + vel_inert_prev*dt + 0.5*accel_inert_prev*dt*dt
    accel_inert = get_force_inertial(vel_inert_prev, q_body, motor_speeds, k_force, k_damp, body_mass)/body_mass
    #print("accel_inert:", accel_inert)
    vel_inert = vel_inert_prev + 0.5*(accel_inert + accel_inert_prev)*dt
    
    return pos_inert, vel_inert, accel_inert

In [12]:
class PID(object):
    def __init__(self, kp, ki, kd, out_max=-np.inf, out_min=np.inf):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.out_max = out_max
        self.out_min = out_min
        self.error_prev = 0.
        self.error_integ = 0.
        self.updated = False
        
    def update(self, sig_in, sig_set):
        error = sig_in - sig_set
        if not self.updated:
            sig_out = self.kp*error
            self.error_prev = error
            self.integ_error += error
            self.updated = True
            return sig_out
        
        error_dot = error - self.error_prev
        self.integrate(error)
        sig_out = self.kp*error + self.kd*error_dot + self.ki*self.error_integ
        return self.bound(sig_out)
        
    def bound(self, sig_out):
        return max(
            min(sig_out, self.out_max
            ), self.out_min
        )
    
    def integrate(self, error):
        if self.error_integ <= self.out_max and self.error_integ >= self.out_min:
            self.error_integ += error
            
        return self.error_integ

class AttitudeController(object):
    # Cascaded PID controller
    # theta_cur ___     _____
    #              |___|     |
    #               ___| PID |____
    # theta_set ___|   |_____|    |                    _____
    #                             |__theta_dot_set____|     |
    #                                             ____| PID |___motor_pulse___
    # theta_dot_cur______________________________|    |_____|
    
    def __init__(self, angle_gains, omega_gains, omega_bounds, psi_gains, psi_bounds):
        self.pid_angle   = PID(angle_gains[0], angle_gains[1], angle_gains[2])
        self.pid_omega   = PID(omega_gains[0], omega_gains[1], omega_gains[2], min(omega_bounds), max(omega_bounds))
        self.pid_psi_dot = PID(psi_gains[0], psi_gains[1], psi_gains[2], min(psi_bounds), max(psi_bounds))
    
    def update(self, omega_cur, q_cur, setpoints):
        # omega_cur = [rad/s, rad/s, rad/s]
        # angle_setpoints = [deg_roll, deg_pitch, deg_yaw/s]
        k_p_angle = 2.0
        k_d_angle = 2.0

        phi   = np.arctan2(2*(q_cur[0]*q_cur[1] + q_cur[2]*q_cur[3]), 1. - 2.*(q_cur[1]**2 + q_cur[2]**2))*180/np.pi
        theta = np.arcsin(2*(q_cur[0]*q_cur[2] - q_cur[3]*q_cur[1]))*180/np.pi

        phi_dot   = omega_cur[0]*180/np.pi
        theta_dot = omega_cur[1]*180/np.pi
        psi_dot   = omega_cur[2]*180/np.pi
        
        omega_setpoint = [None, None, None]
        omega_setpoint[0] = self.pid_angle.update(phi, setpoints[0])
        omega_setpoint[1] = self.pid_angle.update(theta, setpoints[1])
        omega_setpoint[2] = self.pid_psi_dot.update(psi_dot, setpoints[2])
        
        roll_command = self.pid_omega.update(phi_dot, setpoints[0])
        pitch_command = self.pid_omega.update(phi_dot, setpoints[1])
        yaw_dot_command = omega_setpoint[2]
        
        return roll_command, pitch_command, yaw_dot_command
    
    # Callable function returns motor pulses
    def __call__(self, omega_cur, q_cur, setpoints, throttle):
        roll_comm, pitch_comm, yaw_dot_comm = self.update(omega_cur, q_cur, setpoints)
        
        # which motors are on the left and which are on the right? goddamnit.
        # Right side: M1, M2
        # Left side: M3, M4
        '''
                   Front
            CW   M3     M2  CCW
                   \   /
                    \_/
            Left    |_|   Right
                    / \
                   /   \
            CCW  M4     M1  CW
                   Back
        '''
        motor_pulses = [throttle,]*4
        motor_pulses[0] += 0
        return motor_pulses

In [13]:
def run_6dof(
    num_timesteps, # number of dt's
    dt,
    pos_init,
    vel_init,
    q_init,
    omega_init,
    controller=(lambda *x: np.array([750,]*4))
):
    body_mass, body_inertia_tensor, arm_lengths = get_body_constants()
    k_force, k_torque = get_dyn_constants(body_mass)
    print("k_force:", k_force)
    print("body_mass:", body_mass)
    print("4*k_force*ms**2:", 4*k_force*(750**2))
    k_damp = 0.1 #swag
    omega_prev = omega_init
    pos_prev = pos_init
    vel_prev = vel_init
    accel_prev = np.array([0., 0., 0.])/body_mass
    q_prev = q_init
    
    pos_cur = None
    vel_cur = None
    accel_cur = None
    q_cur = None
    omega_cur = None
    
    positions = []
    qs = []
    
    for t in range(0, num_timesteps):
        motor_speeds = controller() # Eventually this will receive arguments
        q_cur, omega_cur = update_orientation_body(
            omega_prev,
            q_prev,
            dt,
            arm_lengths,
            k_force,
            k_torque,
            body_inertia_tensor,
            motor_speeds
        )
        pos_cur, vel_cur, accel_cur = update_pos_vel_inertial(
            pos_prev,
            vel_prev,
            accel_prev,
            q_cur,
            motor_speeds,
            k_force,
            k_damp,
            body_mass,
            dt
        )
        pos_prev = pos_cur
        vel_prev = vel_cur
        accel_prev = accel_cur
        q_prev = q_cur
        omega_prev = omega_cur
        positions.append(pos_prev)
        qs.append(q_prev)
        
    return positions, qs

In [14]:
q_init = np.array([1, 0, 0, 0])
omega_init = np.array([0, 0, 0])
pos_init = np.array([0, 0, 10])
vel_init = np.array([0, 0, -0.1])
g = 9.8 # meters/(second*second)
# Some conflation here between the max/min output of the PID controllers
# and the max/min values from setpoints.
controller = AttitudeController([2.0, 0.0, 1.], [2.0, 0.001, 8.], [-300, 300], [2.0, 0., 10.], [-240, 240])

positions, qs = run_6dof(50000, 0.0005, pos_init, vel_init, q_init, omega_init, controller)

k_force: 2.8776e-06
body_mass: 0.66
4*k_force*ms**2: 6.474600000000001


TypeError: __call__() missing 4 required positional arguments: 'omega_cur', 'q_cur', 'setpoints', and 'throttle'

In [ ]:
plt.figure()
plt.plot([q[0] for q in qs])
plt.show()
plt.figure()
plt.plot([p[2] for p in positions])
plt.show()

In [ ]:
#qs

In [ ]:
def wtf(d):
    x_prev = np.array([0., 0., 0.,])
    v_prev = np.array([0., 0., 0.,])
    a_prev = np.array([0., 0., 0.,])

    poss = []
    
    dt = 0.0001
    for t in range(0, 50000):
        x_next = x_prev + v_prev*dt + 0.5*a_prev*dt*dt
        a_next = np.array([0., 0., -9.8]) - d*v_prev
        v_next = v_prev + 0.5*(a_next + a_prev)*dt
        
        x_prev = x_next
        a_prev = a_next
        v_prev = v_next
        
        poss.append(x_prev)
        
    plt.figure()
    plt.plot([p[2] for p in poss])
    plt.show()

In [ ]:
#wtf(1.0)

In [ ]:
#wtf(0.)